In [0]:
%python
dbutils.widgets.text("p_file_date","2024-12-30")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%python
spark.sql("""
          create table if not exists movie_gold.results_movie
          (
              year_release_date int,
              country_name string,
              company_name string,
              budget float,
              revenue float,
              movie_id int,
              country_id int,
              company_id int,
              created_date date,
              updated_date date
          )
          using delta
          """)

In [0]:
--drop table movie_gold.results_movie

In [0]:
 select * from movie_gold.results_movie

In [0]:
%python
spark.sql(f"""       
            CREATE OR REPLACE TEMP VIEW v_results_movie
            AS
            select M.year_release_date, C.country_name, PCO.company_name,
                M.budget, M.revenue, M.movie_id,C.country_id,PCO.company_id
            FROM movie_silver.movies M 
            INNER JOIN movie_silver.production_country PC ON PC.movie_id = M.movie_id
            INNER JOIN movie_silver.countries C ON C.country_id = PC.country_id
            INNER JOIN movie_silver.movie_company MC ON MC.movie_id = M.movie_id
            INNER JOIN movie_silver.production_company PCO ON PCO.company_id = MC.company_id
            where M.file_date = '{v_file_date}'
            """)


In [0]:
select * from v_results_movie;

In [0]:
%python
spark.sql(f"""       
            MERGE INTO movie_gold.results_movie tgt
            USING v_results_movie src
            ON (tgt.movie_id = src.movie_id and tgt.country_id = src.country_id and tgt.company_id = src.company_id)
            WHEN MATCHED THEN
            UPDATE SET
                tgt.year_release_date = src.year_release_date,
                tgt.country_name = src.country_name,
                tgt.company_name = src.company_name,
                tgt.budget = src.budget,
                tgt.revenue = src.revenue,
                tgt.updated_date = current_timestamp
            WHEN NOT MATCHED THEN
            INSERT      (year_release_date, country_name, company_name, budget, revenue, movie_id, country_id, company_id, created_date)
            VALUES      (year_release_date, country_name, company_name, budget, revenue, movie_id, country_id, company_id, current_timestamp)
  """)

In [0]:
%sql
select created_date,count(*) 
from movie_gold.results_movie
group by created_date